In [ ]:
import cv2
import numpy as np
import urllib.request
import pygame

# Initialize Pygame mixer
pygame.mixer.init()

# Focal length finder function
def FocalLength(measured_distance, real_width, width_in_rf_image):
    focal_length = (width_in_rf_image * measured_distance) / real_width
    return focal_length

# Distance estimation function
def Distance_finder(Focal_Length, real_face_width, face_width_in_frame):
    distance = (real_face_width * Focal_Length) / face_width_in_frame
    return distance

url = 'http://192.168.137.181/cam-hi.jpg'

whT = 320
confThreshold = 0.5
nmsThreshold = 0.3
classesfile = 'coco.names'
classNames = []

with open(classesfile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

modelConfig = 'yolov3.cfg'
modelWeights = 'yolov3.weights'
net = cv2.dnn.readNetFromDarknet(modelConfig, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Define the reference point (assuming it's the center of the image)
ref_point = (whT // 2, whT // 2)

# Focal length and distance calculation parameters
measured_distance = 10  # Measured distance from the camera to the object (in inches)
real_object_width = 10  # Real width of the object (in inches)
width_in_rf_image = 200  # Width of the object in the image (in pixels)

def findObject(outputs, im):
    hT, wT, cT = im.shape
    bbox = []
    classIds = []
    confs = []

    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]

            if confidence > confThreshold:
                w, h = int(det[2] * wT), int(det[3] * hT)
                x, y = int((det[0] * wT) - w / 2), int((det[1] * hT) - h / 2)
                bbox.append([x, y, w, h])
                classIds.append(classId)
                confs.append(float(confidence))

    indices = cv2.dnn.NMSBoxes(bbox, confs, confThreshold, nmsThreshold)

    for i in indices:
        box = bbox[i]
        x, y, w, h = box[0], box[1], box[2], box[3]

        # Calculate distance from the reference point (center of the image) to the detected object
        distance = Distance_finder(focal_length, real_object_width, w) * 10

        # Display distance and name inside the top-left corner of the bounding box
        text = f'{classNames[classIds[i]].upper()} {int(confs[i]*100)}%, {distance:.2f} inches'
        text_width, text_height = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
        cv2.rectangle(im, (x, y - text_height), (x + text_width, y), (255, 255, 255), cv2.FILLED)
        cv2.putText(im, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

        # Define the threshold distance (in inches)
        threshold_distance_inches = 30

        if distance < threshold_distance_inches:
            # Play the sound
            pygame.mixer.Sound('code-red.mp3').play()

        cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 255), 2)

# Calculate the focal length
focal_length = FocalLength(measured_distance, real_object_width, width_in_rf_image)


# # Define the threshold distance (in inches)
# threshold_distance_inches = 20

while True:
    img_resp = urllib.request.urlopen(url)
    imgnp = np.array(bytearray(img_resp.read()), dtype=np.uint8)
    im = cv2.imdecode(imgnp, -1)
    blob = cv2.dnn.blobFromImage(im, 1 / 255, (whT, whT), [0, 0, 0], 1, crop=False)
    net.setInput(blob)
    layernames = net.getLayerNames()
    outputNames = [layernames[i - 1] for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(outputNames)
    findObject(outputs, im)
    cv2.imshow('IMage', im)
    cv2.waitKey(1)
